In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.spatial import distance
import tensorflow as tf
from sklearn.model_selection import train_test_split
import dama as dm
import pickle
import time
import gc
import sys
import awkward as ak

from freedom.toy_model.toy_model_functions import toy_model
from freedom.toy_model.detectors import get_spherical_detector
from types import SimpleNamespace
from freedom.toy_model import NNs

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16 
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 14

# Parameter names
par_names = ['x', 'y', 'z', 't', 'azi', 'zen', 'E', 'trck_frac']

# adds the true parameter values in a plot
def plot_truth(axes, truth, idx=(0,1)):
    if not isinstance(axes, np.ndarray):
        axes = np.array([axes])
    for ax in axes.flatten():
        ax.plot([truth[idx[0]]], [truth[idx[1]]], marker='$T$', markersize=10, color='k')

# plots difference between likelihood scans
def plot_diff(a, b, axes, title_a='a', title_b='b', vmax=None, limit_diff=False, **kwargs):
    
    levels = stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2    
    labels = [str(i) + r'$\sigma$' for i in range(1,6)]
    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
    colors = plt.cm.viridis(np.linspace(0, 0.9, 6))
    a.plot_contour(ax=axes[0], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[0].set_title(title_a)
    b.plot_contour(ax=axes[1], levels=levels,  labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[1].set_title(title_b)
    diff = a - b
    if limit_diff:
        diff.plot(ax=axes[2], cmap='RdBu', cbar=True, vmin=-vmax, vmax=vmax, label=r'$\Delta LLH$', **kwargs)
    else:
        diff.plot(ax=axes[2], cmap='RdBu', cbar=True, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), label=r'$\Delta LLH$', **kwargs) 
    axes[2].set_title(title_a + ' - ' + title_b)

# the highest possible azimuth error is pi
def correct_azi(azi):
    azi = np.where(azi<-np.pi, azi+2*np.pi, azi)
    return np.where(azi>np.pi, azi-2*np.pi, azi)

In [ ]:
#shows the toy detector

detector = get_spherical_detector(radius=10, subdivisions=4)
print(len(detector))

fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(projection='3d')
ax.scatter(detector[:,0], detector[:,1], detector[:,2], color='black', s=50)
ax.grid(False)

In [ ]:
# initialize the toy experiment
toy_experiment = toy_model(detector)

In [ ]:
# generate one test event
truth = np.array([3., 1., 0, 0, 0, np.arccos(1), 5., 0.8])
test_event = toy_experiment.generate_event(truth)

In [ ]:
%%time
# Grid scan

g = dm.GridData(x=np.linspace(-10, 10, 100), y=np.linspace(-10, 10, 100))
IDX = (0,1)

g['dom_hit_term'] = np.empty(g.shape)
g['dom_charge_terms'] = np.empty(g.shape)
g['total_charge_hit_terms'] = np.empty(g.shape)
g['total_charge_terms'] = np.empty(g.shape)

p = np.copy(truth)

for idx in np.ndindex(g.shape):
    p[IDX[0]] = g['x'][idx]
    p[IDX[1]] = g['y'][idx]
    segments = toy_experiment.model(*p)
    g['dom_hit_term'][idx] = toy_experiment.nllh_p_term_dom(segments, test_event[0])
    g['dom_charge_terms'][idx] = toy_experiment.nllh_N_term_dom(segments, test_event[1])
    g['total_charge_hit_terms'][idx] = toy_experiment.nllh_p_term_tot(segments, test_event[0])
    g['total_charge_terms'][idx] = toy_experiment.nllh_N_term_tot(segments, test_event[1])
    
g['dom_hit_term'] -= g['dom_hit_term'].min()
g['dom_charge_terms'] -= g['dom_charge_terms'].min()
g['dom_llh'] = g['dom_hit_term'] + g['dom_charge_terms']
g['total_charge_hit_terms'] -= g['total_charge_hit_terms'].min()
g['total_charge_terms'] -= g['total_charge_terms'].min()
g['total_charge_llh'] = g['total_charge_hit_terms'] + g['total_charge_terms']
g['dom_llh'] -= g['dom_llh'].min()
g['total_charge_llh'] -= g['total_charge_llh'].min()

In [ ]:
# plot the scan
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

plot_diff(g['dom_hit_term'], g['total_charge_hit_terms'], axes=ax[0], title_a='per DOM hit', title_b='total hit', vmax=20, limit_diff=True)
plot_diff(g['dom_charge_terms'], g['total_charge_terms'], axes=ax[1], title_a='per DOM charge', title_b='total charge', vmax=20, limit_diff=True)
plot_diff(g['dom_llh'], g['total_charge_llh'], axes=ax[2], title_a='per DOM llh', title_b='total llh', limit_diff=False)

plot_truth(ax, truth, IDX)

# Train NNs

In [ ]:
# reduce NE for faster (but less precise) results
NE, Set = 1_000_000, 0
events, meta = toy_experiment.generate_events(n=NE, gamma=0, gen_volume="sphere", e_lim=(1,12), inelast_lim=(0,1),
                                              t_width=0, radius=10, contained=True, min_hits=3, rand=Set)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
nGPUs = strategy.num_replicas_in_sync

## Hit Net - total charge

In [ ]:
x, t = NNs.get_hit_data(events)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs, time_spread=10)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs, time_spread=10)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

with strategy.scope():
    hmodel_t = NNs.get_hmodel(x_shape=6, t_shape=8, trafo=NNs.hit_trafo_3D, activation='swish', final_activation='swish')
    hmodel_t.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = hmodel_t.fit(d_train, epochs=15, verbose=1, validation_data=d_valid)

In [ ]:
xxs = np.repeat(test_event[0][np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 6)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(test_event[0]), axis=0)

hmodel_t.layers[-1].activation = tf.keras.activations.linear
hmodel_t.compile()

llhs = -hmodel_t((xxs, tts)).numpy()
llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(test_event[0]))), axis=1)

g.hit_llh_total = llhs.reshape(g.shape)
g.hit_llh_total -= g.hit_llh_total.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.total_charge_hit_terms, g.hit_llh_total, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)

## Charge Net - total charge

In [ ]:
x, t = NNs.get_charge_data(events)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

with strategy.scope():
    cmodel = NNs.get_cmodel(x_shape=2, t_shape=8, trafo=NNs.charge_trafo_3D, activation='swish', final_activation='swish')
    cmodel.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = cmodel.fit(d_train, epochs=100, verbose=1, validation_data=d_valid) #

In [ ]:
xxs = np.tile([len(test_event[0]), len(np.unique(test_event[0][:,0]))], np.prod(g.shape))
xxs = xxs.reshape(-1, 2)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)

cmodel.layers[-1].activation = tf.keras.activations.linear
cmodel.compile()

llhs = np.nan_to_num(-cmodel((xxs, tts)))

g.charge_llh_total = llhs.reshape(g.shape)
g.charge_llh_total -= g.charge_llh_total.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.total_charge_terms, g.charge_llh_total, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)

## LLH - total charge

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_diff(g.total_charge_hit_terms, 
          g.hit_llh_total, 
          title_a='Hit Analytic', title_b='Hit NN', vmax=10, axes=ax[0], limit_diff=True)

plot_diff(g.total_charge_terms, 
          g.charge_llh_total, 
          title_a='Charge Analytic', title_b='Charge NN', vmax=10, axes=ax[1], limit_diff=True)

ana, NN = g.total_charge_hit_terms+g.total_charge_terms, g.hit_llh_total+g.charge_llh_total
plot_diff(ana-ana.min(), 
          NN-NN.min(), 
          title_a='Analytic', title_b='NN', vmax=10, axes=ax[2], limit_diff=True)
plot_truth(ax, truth, IDX)

## Hit Net - per dom

In [ ]:
x, t = NNs.get_hit_data(events)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs, shuffle='inDOM', time_spread=10)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs, shuffle='inDOM', time_spread=10)

In [ ]:
optimizer = tf.keras.optimizers.Adam(2e-5)

with strategy.scope():
    hmodel_d = NNs.get_hmodel(x_shape=6, t_shape=8, trafo=NNs.hit_trafo_3D, activation='swish', final_activation='swish')
    hmodel_d.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = hmodel_d.fit(d_train, epochs=7, verbose=1, validation_data=d_valid)

In [ ]:
xxs = np.repeat(test_event[0][np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 6)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(test_event[0]), axis=0)

hmodel_d.layers[-1].activation = tf.keras.activations.linear
hmodel_d.compile()

llhs = -hmodel_d.predict((xxs, tts), batch_size=4096)

llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(test_event[0]))), axis=1)

g.hit_llh_dom = llhs.reshape(g.shape)

g.hit_llh_dom -= g.hit_llh_dom.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.dom_hit_term, g.hit_llh_dom, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)

## Charge Net - per DOM

In [ ]:
x, t = NNs.get_dom_data(events, detector)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(5e-4)

with strategy.scope():
    dmodel = NNs.get_hmodel(x_shape=4, t_shape=8, trafo=NNs.dom_trafo_3D, activation='swish', final_activation='swish')
    dmodel.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = dmodel.fit(d_train, epochs=5, verbose=1, validation_data=d_valid)

In [ ]:
xx = []
ind = test_event[0][:, 5]
for i in range(len(detector)):
    d = np.append(detector[i], np.sum(ind==i))
    xx.append(list(d))
xxs = np.repeat(np.array(xx)[np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 4)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(detector), axis=0)

dmodel.layers[-1].activation = tf.keras.activations.linear
dmodel.compile()

llhs = -dmodel.predict((xxs, tts), batch_size=4096)

llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(detector))), axis=1)

g.charge_llh_dom = llhs.reshape(g.shape)

g.charge_llh_dom -= g.charge_llh_dom.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.dom_charge_terms, g.charge_llh_dom, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)

## LLH - per DOM

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_diff(g.dom_hit_term, 
          g.hit_llh_dom, 
          title_a='Hit Analytic', title_b='Hit NN', vmax=10, axes=ax[0], limit_diff=True)

plot_diff(g.dom_charge_terms, 
          g.charge_llh_dom, 
          title_a='Charge Analytic', title_b='Charge NN', vmax=10, axes=ax[1], limit_diff=True)

ana, NN = g.dom_hit_term+g.dom_charge_terms, g.hit_llh_dom+g.charge_llh_dom
plot_diff(ana-ana.min(), 
          NN-NN.min(), 
          title_a='Analytic', title_b='NN', vmax=10, axes=ax[2], limit_diff=True)
plot_truth(ax, truth, IDX)

## LLH - both formulations

In [ ]:
# overlays two 2D likelihood scans
def plot_overlay(a, b, ax, **kwargs):
    levels = stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2    
    labels = [str(i) + r'$\sigma$' for i in range(1,6)]
    colors = plt.cm.viridis(np.linspace(0, 0.9, 6))
    a.plot_contour(ax=ax, levels=levels, labels=labels, colors=colors, **kwargs)
    b.plot_contour(ax=ax, levels=levels, linestyles=[':']*len(levels), colors=colors, **kwargs)
    ax.plot([], [], label='Analytic', color='Tab:blue')
    ax.plot([], [], label='NN', linestyle=':', color='Tab:blue')
    ax.set_xlabel(par_names[IDX[0]])
    ax.set_ylabel(par_names[IDX[1]])

In [ ]:
g['llh_dom'] = g['hit_llh_dom'] + g['charge_llh_dom']
g['llh_total'] = g['hit_llh_total'] + g['charge_llh_total']
g['llh_dom'] -= g['llh_dom'].min()
g['llh_total'] -= g['llh_total'].min()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20,12))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_truth(ax, truth, IDX)

ax[0][0].set_title('HitNet', size=20)
plot_overlay(g.dom_hit_term, g.hit_llh_dom, ax[0][0])
ax[0][0].text(g.x.min()-2.5, g.y.mean(), 'Per Sensor', rotation=90, size=20, ha='center', va='center')

ax[0][1].set_title('ChargeNet', size=20)
plot_overlay(g.dom_charge_terms, g.charge_llh_dom, ax[0][1])

ax[0][2].set_title('Complete LLH', size=20)
plot_overlay(g.dom_llh, g.llh_dom, ax[0][2])
ax[0][2].legend(loc='upper left')

plot_overlay(g.total_charge_hit_terms, g.hit_llh_total, ax[1][0])
ax[1][0].text(g.x.min()-2.5, g.y.mean(), 'Total Detector', rotation=90, size=20, ha='center', va='center')

plot_overlay(g.total_charge_terms, g.charge_llh_total, ax[1][1])

plot_overlay(g.total_charge_llh, g.llh_total, ax[1][2])

# Event reweighting

In [ ]:
def get_hit_data(truth, vec, bins):
    t = truth.reshape((1,8))
    ts = np.repeat(t, len(hits), axis=0)

    r_t = np.exp(hmodel_t.predict([hits, ts], batch_size=5000)).flatten()
    r_d = np.exp(hmodel_d.predict([hits, ts], batch_size=5000)).flatten()

    dists = distance.cdist(detector[:,:3], toy_experiment.model(*truth)[:,:3])
    survive = toy_experiment.survival(dists)
    hit_llh_p, hit_llh_ps = [], []
    for b in bins:
        mat = toy_experiment.pandel.pdf(b-dists*4.333, d=dists)
        hit_llh_p.append(np.sum(np.sum(mat, axis=0) * vec))
        hit_llh_ps.append(np.sum(np.sum(mat * survive, axis=0) * vec))
    norm_p = np.sum(np.array(hit_llh_p)) * np.diff(bins)[0]
    norm_ps = np.sum(np.array(hit_llh_ps)) * np.diff(bins)[0]
    
    return r_t, r_d, hit_llh_p, hit_llh_ps, norm_p, norm_ps

def get_charge_data(truth, exp_bins, exp_bins_fine):
    t = truth.reshape((1,8))
    ts_t = np.repeat(t, len(charges_t), axis=0)
    ts_d = np.repeat(t, len(charges_d), axis=0)

    r_t = np.exp(cmodel.predict([charges_t, ts_t], batch_size=5000)).flatten()
    r_d = np.exp(dmodel.predict([charges_d, ts_d], batch_size=5000)).flatten()

    N_exp = toy_experiment.N_exp(toy_experiment.model(*truth))
    dom_c_llh = np.zeros(len(exp_bins))
    for N in N_exp:
        dom_c_llh += stats.poisson.pmf(exp_bins, mu=N)
        
    return r_t, r_d, N_exp, dom_c_llh

In [ ]:
events, meta = toy_experiment.generate_events(n=100_000, gamma=0, gen_volume="sphere", e_lim=(1,12), inelast_lim=(0,1),
                                              t_width=0, radius=10, contained=True, min_hits=3)
truths = NNs.make_truth_array(events)

### hitNets

In [ ]:
%%time
hits = NNs.get_hit_data(events)[0]
hits[:, 3] += np.random.normal(0, 10, len(hits))

In [ ]:
bins = np.linspace(-20,200,100)

r_t, r_d, hit_llh_p, hit_llh_ps, norm_p, norm_ps = get_hit_data(
                                                            np.array([3., 1., 0, 0, 0, np.arccos(1), 5., 0.8]), 
                                                            [4,0.2,0.2,0.2,0.2,0.2],
                                                            bins)

r_t2, r_d2, hit_llh_p2, hit_llh_ps2, norm_p2, norm_ps2 = get_hit_data(
                                                            np.array([-5., 7., 0, 0, 0, np.arccos(1), 3., 1]), 
                                                            [3],
                                                            bins)

r_t3, r_d3, hit_llh_p3, hit_llh_ps3, norm_p3, norm_ps3 = get_hit_data(
                                                            np.array([-5., 3., 0, 0, 0, np.arccos(1), 4., 0.9]), 
                                                            [3.6,0.2,0.2],
                                                            bins)

In [ ]:
plt.figure(figsize=(14,7*0.618))

plt.subplot(121)
plt.hist(hits[:, 3], bins, label=r'$p(x)$', density=True, histtype='step', color='black')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_ps)/norm_ps, c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_t2, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_ps2)/norm_ps2, c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_t3, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_ps3)/norm_ps3, c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.legend()
plt.title('All-sensor')
plt.xlabel(r'$x$ = Hit time')
plt.ylabel('PDF')


plt.subplot(122)
plt.hist(hits[:, 3], bins, label=r'$p(x)$', density=True, histtype='step', color='black')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_p)/norm_p, c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_p2)/norm_p2, c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_d3, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_p3)/norm_p3, c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.legend()
plt.title('Per-sensor')
plt.xlabel(r'$x$ = Hit time')

### chargeNets

In [ ]:
%%time
charges_t = NNs.get_charge_data(events)[0]
charges_d = NNs.get_dom_data(events, detector)[0]

In [ ]:
exp_bins, exp_bins_fine = np.linspace(0,12,13), np.linspace(15,150,136)

r_t, r_d, N_exp, dom_c_llh = get_charge_data(np.array([3., 1., 0, 0, 0, np.arccos(1), 5., 0.8]),
                                             exp_bins, exp_bins_fine)
r_t2, r_d2, N_exp2, dom_c_llh2 = get_charge_data(np.array([-1., 2., 0, 0, 0, np.arccos(1), 2., 1]),
                                                 exp_bins, exp_bins_fine)
r_t3, r_d3, N_exp3, dom_c_llh3 = get_charge_data(np.array([-7., 1., 0, 0, 0, np.arccos(0), 5., 0.8]),
                                                 exp_bins, exp_bins_fine)

In [ ]:
plt.figure(figsize=(14,7*0.618))

plt.subplot(121)
bins = np.linspace(15,149,68)
plt.hist(charges_t[:, 0], bins, label='p(x)', density=True, histtype='step', color='black')

plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t, density=True, histtype='step')
plt.plot(exp_bins_fine+np.diff(exp_bins_fine)[0]/2, stats.poisson.pmf(exp_bins_fine, mu=np.sum(N_exp)), 
         c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_t2, density=True, histtype='step')
plt.plot(exp_bins_fine+np.diff(exp_bins_fine)[0]/2, stats.poisson.pmf(exp_bins_fine, mu=np.sum(N_exp2)), 
         c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_t3, density=True, histtype='step')
plt.plot(exp_bins_fine+np.diff(exp_bins_fine)[0]/2, stats.poisson.pmf(exp_bins_fine, mu=np.sum(N_exp3)), 
         c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.title('All-sensor')
plt.legend()
plt.xlabel(r'$x$ = Charge (total detector)')
plt.ylabel('PDF')


plt.subplot(122)
bins = np.linspace(0,12,13)
plt.hist(charges_d[:, 3], bins, label='p(x)', density=True, histtype='step', color='black')

plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d, density=True, histtype='step')
plt.plot(exp_bins+np.diff(exp_bins)[0]/2, dom_c_llh/len(N_exp), c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2, density=True, histtype='step')
plt.plot(exp_bins+np.diff(exp_bins)[0]/2, dom_c_llh2/len(N_exp2), c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_d3, density=True, histtype='step')
plt.plot(exp_bins+np.diff(exp_bins)[0]/2, dom_c_llh3/len(N_exp3), c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.title('Per-sensor')
plt.legend()
plt.yscale('log')
plt.ylim(1e-4, 2)
plt.xlabel(r'$x$ = Charge (sensor)')

In [ ]:
bins = np.linspace(0,13,14)
plt.figure(figsize=(16,12))
plt.subplots_adjust(wspace=0.25, hspace=0.3)

for i, n in enumerate([0,20,40,45,50,70,100,120,150]):
    plt.subplot(3,3,i+1)
    idx = len(detector)*np.array(range(len(events)))+n
    plt.hist(charges_d[idx, 3], bins, label='p(x)', density=True, histtype='step', color='black')
    
    plt.hist(charges_d[idx, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d[idx], density=True, histtype='step')
    plt.plot(exp_bins+np.diff(exp_bins)[0]/2, stats.poisson.pmf(exp_bins, mu=N_exp[n]), 
             c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')
    
    plt.hist(charges_d[idx, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2[idx], density=True, histtype='step')
    plt.plot(exp_bins+np.diff(exp_bins)[0]/2, stats.poisson.pmf(exp_bins, mu=N_exp2[n]), 
             c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')
    
    plt.hist(charges_d[idx, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_d3[idx], density=True, histtype='step')
    plt.plot(exp_bins+np.diff(exp_bins)[0]/2, stats.poisson.pmf(exp_bins, mu=N_exp3[n]), 
             c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')
    
    plt.title('DOM '+str(n))
    if i == 2: plt.legend()
    plt.yscale('log')
    if i > 5: plt.xlabel(r'$x$ = Charge (sensor)')
    if i%3 == 0: plt.ylabel('PDF')
    plt.ylim(1e-6, 1e0)